In [ ]:
# %pip install openai anthropic streamlit python-dotenv sec-api

In [65]:
import os
import json
import openai
import anthropic
import pandas as pd
import streamlit as st
from dotenv import load_dotenv
from sec_api import FormAdvApi, ExtractorApi

load_dotenv()
pd.set_option('display.max_columns', None)

openai.api_key = st.secrets['OPENAI_API_KEY']
form_adv_api = FormAdvApi(api_key=st.secrets['SEC_API_KEY'])
extractor_api = ExtractorApi(api_key=st.secrets['SEC_API_KEY'])
client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

In [72]:
form_adv_filings_df = pd.read_csv(r"data\clean\form_adv_filings_df.csv", low_memory=False).set_index('BusNm')
form_adv_labeled_PROPERLY = pd.read_csv(r"data\labeled\form_adv_labeled_PROPERLY.csv", low_memory=False)
map_definitions = pd.read_csv(r"data\mapping\map_definitions.csv", low_memory=False)
form_adv_filings_json = r"data\json\form_adv_filings.json"

with open(r"data\json\form_adv_filings.json", 'r') as file:
    form_adv_filings_json = json.load(file)
    
with open(r"data\json\form_adv_filings_unicode.json", 'r', encoding='utf-8') as file:
    form_adv_filings_json_utf = json.load(file)    
    
with open(r'data/json/map_definitions.json') as f:
    labels = json.load(f)

In [73]:
df = pd.read_excel(r"C:\Users\gpisciotta\Documents\Book2.xlsx")

# Combine non-null values in each row into a single string (excluding the 'filings' column)
df['Description'] = df.drop(columns=['filings']).apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)

# Keep only the 'filings' and 'Description' columns and create a copy
df2_clean = df[['filings', 'Description']].copy()

# Remove everything after a period (.) in the 'Description' column
df2_clean['Description'] = df2_clean['Description'].apply(lambda x: x.split(".")[0].strip())

df2_clean = df2_clean[df2_clean['Description'] != ""]

# Display the cleaned DataFrame
print(df2_clean.shape)
df2_clean.head()

(381, 2)


,filings,Description
1,SECRgnCD,Business type
2,FirmCrdNb,"CRD number assigned by the FINRA's CRD system or by the IARD system, e"
3,SECNb,Firm's 801# or 802# from the IARD composite record
4,BusNm,Firm's primary business name from the IARD composite record
5,LegalNm,Firm's legal name from the IARD composite record


In [74]:
current_cols_list = form_adv_filings_df.reset_index().columns.to_list()

In [75]:
pd.set_option('display.max_colwidth', None)

df2_clean = df2_clean[df2_clean['filings'].isin(current_cols_list)]
df2_clean

,filings,Description
1,SECRgnCD,Business type
2,FirmCrdNb,"CRD number assigned by the FINRA's CRD system or by the IARD system, e"
3,SECNb,Firm's 801# or 802# from the IARD composite record
4,BusNm,Firm's primary business name from the IARD composite record
5,LegalNm,Firm's legal name from the IARD composite record
...,...,...
399,Q1BJ2BChfc,"Firm's most recent response to Part 1B Item 2J(2)(b)(3) Chartered Financial Consultant (""ChFC"")"
400,Q1BJ2BCic,"Firm's most recent response to Part 1B Item 2J(2)(b)(4) Chartered Investment Counselor (""CIC"")"
401,Q1BJ2BPfs,"Firm's most recent response to Part 1B Item 2J(2)(b)(5) Personal Financial Specialist (""PFS"")"
402,Q1BJ2BNone,Firm's most recent response to Part 1B Item 2J(2)(b)(6) None of the above


In [76]:
# Assuming df1 is the DataFrame with 'filings' and 'Description' columns
# and df2 is the DataFrame whose column names you want to replace

# Create a dictionary from df1
column_name_dict = df2_clean.set_index('filings')['Description'].to_dict()

# Make a copy of the DataFrame
form_adv_filings_df = form_adv_filings_df.reset_index().copy()

# Rename the columns of df2
form_adv_filings_df.rename(columns=column_name_dict, inplace=True)
form_adv_filings_df

,Firm's primary business name from the IARD composite record,Filing_Dt,Filing_FormVrsn,Rgstn_FirmType,Rgstn_St,Rgstn_Dt,States_RgltrCd,States_St,States_Dt,id,Business type,"CRD number assigned by the FINRA's CRD system or by the IARD system, e",Firm's 801# or 802# from the IARD composite record,Firm's legal name from the IARD composite record,Flag indicating if this firm registers more than 1 investment adviser under an umbrella registration,Strt1,City,State,Cntry,PostlCd,PhNb,FaxNb,WebAddr,The total number of offices other than the principal office and place of business,Firm's response to Part1A Item 1I: Do you have one or more websites (Y/N),Firm's response to Part1A Item 1M: Are you registered with a foreign financial regulatory authority (Y/N),Firm's response to Part1A Item 1N: Are you a public reporting company under Sections 12 or 15(d) of the Securities Exchange Act of 1934 (Y/N),Firm's response to Part1A Item 1O: Did you have 1 billion or more in assets on the last day of your most recent fiscal year (Y/N),Firm's response to Part1A Item 2A(1): are you a large advisory firm that either: (a) has regulatory assets under management of 100 million (in U,Firm's response to Part1A Item 2A(2): are you a mid-sized advisory firm that has regulatory assets under management of 25 million (in U,"Firm's response to Part1A Item 2A(4): is your principal office and place of business outside the United States (Y/N)? Allowed values: Y = yes, N = is not","Firm's response to Part1A Item 2A(5): are you an investment adviser (or sub-adviser) to an investment company registered under the Investment Company Act of 1940 Allowed values: Y = yes, N = is not","Firm's response to Part1A Item 2A(6): are you an investment adviser to a company which has elected to be a business development company pursuant to section 54 of the Investment Company Act of 1940 and has not withdrawn the election, and you have at least 25 million of regulatory assets under management",Firm's response to Part1A Item 2A(7): are yo a pension consultant with respect to assets of plans having an aggregate value of at least 200000000 that qualifies for the exemption in rule 203A-2,"Firm's response to Part1A Item 2A(8): are you a related adviser under rule 203A-2(b) that controls, is controlled by, or is under common control with, an investment adviser that is registered with the SEC, and your principal office and place of business is the same as the registered adviser",Firm's response to Part1A Item 2A(9): are you a newly formed adviser relying on rule 203A-2(c) because you expect to be eligible for SEC registration within 120 days,Firm's response to Part1A Item 2A(10) are you a multi-state adviser that is required to register in 15 or more states and is relying on rule 203A-2(d),Firm's response to Part1A Item 2A(11): are you an Internet adviser relying on rule 203A-2(e),Firm's response to Part1A Item 2A(12): have received an SEC order exempting you from the prohibition against registration with the SEC,Firm's response to Part1A Item 2A(13): are no longer eligible to remain registered with the SEC,Firm's response to Part1A Item 3A: How are you organized (description),Firm's response to Part 1A Item 3B: In what month does your fiscal year end each year?,State code,Country,TtlEmp,Firm's response to Part1A Item 5B(1): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(2): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(3): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(4): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(5): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(6): Approximately how many firms or other persons solicit advisory clients on your behalf,Firm's response to Part1A Item 5C(1): To approximately how many clients did you provide investment advisory services during your m

In [77]:
form_adv_filings_df[form_adv_filings_df["Firm's primary business name from the IARD composite record"].str.contains('cresset', case=False)]

,Firm's primary business name from the IARD composite record,Filing_Dt,Filing_FormVrsn,Rgstn_FirmType,Rgstn_St,Rgstn_Dt,States_RgltrCd,States_St,States_Dt,id,Business type,"CRD number assigned by the FINRA's CRD system or by the IARD system, e",Firm's 801# or 802# from the IARD composite record,Firm's legal name from the IARD composite record,Flag indicating if this firm registers more than 1 investment adviser under an umbrella registration,Strt1,City,State,Cntry,PostlCd,PhNb,FaxNb,WebAddr,The total number of offices other than the principal office and place of business,Firm's response to Part1A Item 1I: Do you have one or more websites (Y/N),Firm's response to Part1A Item 1M: Are you registered with a foreign financial regulatory authority (Y/N),Firm's response to Part1A Item 1N: Are you a public reporting company under Sections 12 or 15(d) of the Securities Exchange Act of 1934 (Y/N),Firm's response to Part1A Item 1O: Did you have 1 billion or more in assets on the last day of your most recent fiscal year (Y/N),Firm's response to Part1A Item 2A(1): are you a large advisory firm that either: (a) has regulatory assets under management of 100 million (in U,Firm's response to Part1A Item 2A(2): are you a mid-sized advisory firm that has regulatory assets under management of 25 million (in U,"Firm's response to Part1A Item 2A(4): is your principal office and place of business outside the United States (Y/N)? Allowed values: Y = yes, N = is not","Firm's response to Part1A Item 2A(5): are you an investment adviser (or sub-adviser) to an investment company registered under the Investment Company Act of 1940 Allowed values: Y = yes, N = is not","Firm's response to Part1A Item 2A(6): are you an investment adviser to a company which has elected to be a business development company pursuant to section 54 of the Investment Company Act of 1940 and has not withdrawn the election, and you have at least 25 million of regulatory assets under management",Firm's response to Part1A Item 2A(7): are yo a pension consultant with respect to assets of plans having an aggregate value of at least 200000000 that qualifies for the exemption in rule 203A-2,"Firm's response to Part1A Item 2A(8): are you a related adviser under rule 203A-2(b) that controls, is controlled by, or is under common control with, an investment adviser that is registered with the SEC, and your principal office and place of business is the same as the registered adviser",Firm's response to Part1A Item 2A(9): are you a newly formed adviser relying on rule 203A-2(c) because you expect to be eligible for SEC registration within 120 days,Firm's response to Part1A Item 2A(10) are you a multi-state adviser that is required to register in 15 or more states and is relying on rule 203A-2(d),Firm's response to Part1A Item 2A(11): are you an Internet adviser relying on rule 203A-2(e),Firm's response to Part1A Item 2A(12): have received an SEC order exempting you from the prohibition against registration with the SEC,Firm's response to Part1A Item 2A(13): are no longer eligible to remain registered with the SEC,Firm's response to Part1A Item 3A: How are you organized (description),Firm's response to Part 1A Item 3B: In what month does your fiscal year end each year?,State code,Country,TtlEmp,Firm's response to Part1A Item 5B(1): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(2): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(3): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(4): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(5): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(6): Approximately how many firms or other persons solicit advisory clients on your behalf,Firm's response to Part1A Item 5C(1): To approximately how many clients did you provide investment advisory services during your m

In [78]:
form_adv_filings_df.set_index("Firm's primary business name from the IARD composite record").to_csv("C:/Users/gpisciotta/gdp_local_code/crestone_capital/competitor_profile/data/live/form_adv_filing_mapped_df.csv")

In [80]:
competitor_list = [
    'CRESTONE ASSET MANAGEMENT LLC', 
    'CRESSET ASSET MANAGEMENT, LLC', 
    'CRESSET PARTNERS LLC',
    'PROVENIO CAPITAL', 
    'JOHNSON FINANCIAL GROUP LLC',
    'SEVEN POST INVESTMENT OFFICE LP',
    'BBR PARTNERS, LLC',
    'GRESHAM INVESTMENT MANAGEMENT LLC',
    'HIRTLE, CALLAGHAN & CO., LLC'
]

competitor_df = form_adv_filings_df[form_adv_filings_df["Firm's primary business name from the IARD composite record"].isin(competitor_list)]

competitor_df.to_csv(r"C:\Users\gpisciotta\gdp_local_code\crestone_capital\competitor_profile\data\live\competitor_df.csv")

competitor_df

,Firm's primary business name from the IARD composite record,Filing_Dt,Filing_FormVrsn,Rgstn_FirmType,Rgstn_St,Rgstn_Dt,States_RgltrCd,States_St,States_Dt,id,Business type,"CRD number assigned by the FINRA's CRD system or by the IARD system, e",Firm's 801# or 802# from the IARD composite record,Firm's legal name from the IARD composite record,Flag indicating if this firm registers more than 1 investment adviser under an umbrella registration,Strt1,City,State,Cntry,PostlCd,PhNb,FaxNb,WebAddr,The total number of offices other than the principal office and place of business,Firm's response to Part1A Item 1I: Do you have one or more websites (Y/N),Firm's response to Part1A Item 1M: Are you registered with a foreign financial regulatory authority (Y/N),Firm's response to Part1A Item 1N: Are you a public reporting company under Sections 12 or 15(d) of the Securities Exchange Act of 1934 (Y/N),Firm's response to Part1A Item 1O: Did you have 1 billion or more in assets on the last day of your most recent fiscal year (Y/N),Firm's response to Part1A Item 2A(1): are you a large advisory firm that either: (a) has regulatory assets under management of 100 million (in U,Firm's response to Part1A Item 2A(2): are you a mid-sized advisory firm that has regulatory assets under management of 25 million (in U,"Firm's response to Part1A Item 2A(4): is your principal office and place of business outside the United States (Y/N)? Allowed values: Y = yes, N = is not","Firm's response to Part1A Item 2A(5): are you an investment adviser (or sub-adviser) to an investment company registered under the Investment Company Act of 1940 Allowed values: Y = yes, N = is not","Firm's response to Part1A Item 2A(6): are you an investment adviser to a company which has elected to be a business development company pursuant to section 54 of the Investment Company Act of 1940 and has not withdrawn the election, and you have at least 25 million of regulatory assets under management",Firm's response to Part1A Item 2A(7): are yo a pension consultant with respect to assets of plans having an aggregate value of at least 200000000 that qualifies for the exemption in rule 203A-2,"Firm's response to Part1A Item 2A(8): are you a related adviser under rule 203A-2(b) that controls, is controlled by, or is under common control with, an investment adviser that is registered with the SEC, and your principal office and place of business is the same as the registered adviser",Firm's response to Part1A Item 2A(9): are you a newly formed adviser relying on rule 203A-2(c) because you expect to be eligible for SEC registration within 120 days,Firm's response to Part1A Item 2A(10) are you a multi-state adviser that is required to register in 15 or more states and is relying on rule 203A-2(d),Firm's response to Part1A Item 2A(11): are you an Internet adviser relying on rule 203A-2(e),Firm's response to Part1A Item 2A(12): have received an SEC order exempting you from the prohibition against registration with the SEC,Firm's response to Part1A Item 2A(13): are no longer eligible to remain registered with the SEC,Firm's response to Part1A Item 3A: How are you organized (description),Firm's response to Part 1A Item 3B: In what month does your fiscal year end each year?,State code,Country,TtlEmp,Firm's response to Part1A Item 5B(1): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(2): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(3): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(4): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(5): Approximately how many of the employees reported in 5,Firm's response to Part1A Item 5B(6): Approximately how many firms or other persons solicit advisory clients on your behalf,Firm's response to Part1A Item 5C(1): To approximately how many clients did you provide investment advisory services during your m